In [2]:
import os
import gc
import numpy as np
import pandas as pd
import xgboost as xgb
import joblib
import optuna
import category_encoders as ce
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import f1_score
from imblearn.over_sampling import SMOTE
import warnings

warnings.filterwarnings('ignore')

c:\Users\CY2\miniconda3\envs\DS\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import os
import pandas as pd
import gc

# ==========================================
# 1. 경로 설정 (Train -> Test로 변경)
# ==========================================
# 각 폴더명 딕셔너리 (동일함)
folder_name = {1:"1.회원정보", 2:"2.신용정보", 3:"3.승인매출정보", 4:"4.청구입금정보", 
               5:"5.잔액정보", 6:"6.채널정보", 7:"7.마케팅정보", 8:"8.성과정보"}

# 현재 디렉터리
HOME = os.getcwd()

# 폴더 리스트 생성 (경로를 'train'에서 'test'로 변경)
folder_list = []
for value in folder_name.values():
    # ⚠️ 중요: 경로가 data/test 인지 확인하세요
    folder_list.append(os.path.join(HOME, "test", value)) 

print(f"총 폴더 개수: {len(folder_list)}")

# 폴더 내 파일 리스트 읽어오기
fileNameList = {}
for i in range(len(folder_list)):
    # ⚠️ 파일 순서가 섞이지 않도록 sorted() 추가 (7월, 8월... 순서 보장)
    fileNameList[i+1] = sorted(os.listdir(folder_list[i]))
    
print(f"첫 번째 폴더 파일 목록 예시: {fileNameList[1]}")

# 파일 경로 리스트 생성
filePathList = {}
temp = []
for i in range(len(folder_list)):
    for j in range(len(fileNameList[i+1])):
        temp.append(os.path.join(folder_list[i], fileNameList[i+1][j]))
    filePathList[i+1] = temp
    temp = []

# ==========================================
# 2. 메모리 최적화 및 병합 함수
# ==========================================
def optimize_memory(df):
    for col in df.select_dtypes(include=['number']).columns:
        if df[col].dtype == 'int64':
            df[col] = pd.to_numeric(df[col], downcast='integer')
        elif df[col].dtype == 'float64':
            df[col] = pd.to_numeric(df[col], downcast='float')
    return df

def create_Test_DataFrame(filePathList, month):
    # month-7 인덱싱을 위해 7~12 범위 확인
    if 7 <= month <= 12:
        print(f"{month}월 데이터 병합 시작...")
        for i in range(8): # 1번~8번 폴더 순회
            
            # 파일 경로 가져오기 (month-7 인덱스 사용)
            # 예: 7월 -> index 0, 8월 -> index 1
            current_file_path = filePathList[i+1][month-7]
            
            # 단계 1: 읽기 및 최적화
            # engine='fastparquet'가 없으면 pyarrow 등 기본 엔진 사용
            try:
                df = pd.read_parquet(current_file_path, engine="fastparquet")
            except:
                df = pd.read_parquet(current_file_path) # 엔진 지정 없이 재시도
                
            df = optimize_memory(df)
            
            # 단계 2: 병합
            if i == 0:
                temp_df = df.copy()
            else:
                # 기준년월, ID 기준으로 Left Join
                temp_df = pd.merge(temp_df, df, on=["기준년월", "ID"], how="left")
        
        print(f"✅ {month}월 병합 완료! 크기: {temp_df.shape}")
        return temp_df
    else:
        print("Error: Month must be between 7 and 12")
        return None

# ==========================================
# 3. 실행 및 저장
# ==========================================

# 7월부터 12월까지 Test 데이터 생성 및 저장
months = range(7, 13)

for m in months:
    # 1. 병합
    df_test_month = create_Test_DataFrame(filePathList, m)
    
    # 2. 저장 (파일명: credit_test_month_X.parquet)
    save_name = f"credit_test_month_{m}.parquet"
    df_test_month.to_parquet(save_name)
    print(f"💾 {save_name} 저장 완료.\n")
    
    # 3. 메모리 해제
    del df_test_month
    gc.collect()

print("🎉 모든 Test 데이터 처리 완료!")

총 폴더 개수: 8
첫 번째 폴더 파일 목록 예시: ['201807_test_회원정보.parquet', '201808_test_회원정보.parquet', '201809_test_회원정보.parquet', '201810_test_회원정보.parquet', '201811_test_회원정보.parquet', '201812_test_회원정보.parquet']
7월 데이터 병합 시작...
✅ 7월 병합 완료! 크기: (100000, 857)
💾 credit_test_month_7.parquet 저장 완료.

8월 데이터 병합 시작...
✅ 8월 병합 완료! 크기: (100000, 857)
💾 credit_test_month_8.parquet 저장 완료.

9월 데이터 병합 시작...
✅ 9월 병합 완료! 크기: (100000, 857)
💾 credit_test_month_9.parquet 저장 완료.

10월 데이터 병합 시작...
✅ 10월 병합 완료! 크기: (100000, 857)
💾 credit_test_month_10.parquet 저장 완료.

11월 데이터 병합 시작...
✅ 11월 병합 완료! 크기: (100000, 857)
💾 credit_test_month_11.parquet 저장 완료.

12월 데이터 병합 시작...
✅ 12월 병합 완료! 크기: (100000, 857)
💾 credit_test_month_12.parquet 저장 완료.

🎉 모든 Test 데이터 처리 완료!


In [4]:
# =========================================================
# ✅ Step 1: 월별 파일(7~12월) 병합
# =========================================================
FILE_PATH = r"C:\Users\CY2\github\DataScience\Credit Card"
months = range(7, 13)
train_list = []

print("🔄 월별 파일 병합 시작 (7월~12월)...")
for m in months:
    file_name = f"credit_preprocessing_month_{m}.parquet"
    full_path = os.path.join(FILE_PATH, file_name)
    if os.path.exists(full_path):
        df = pd.read_parquet(full_path)
        train_list.append(df)
        print(f"   - {file_name} 로드 완료")
    else:
        print(f"   ⚠️ 파일 없음: {file_name}")

# 전체 병합 후 리스트 삭제 (메모리 확보)
train = pd.concat(train_list, ignore_index=True)
del train_list
gc.collect()
print(f"✅ 병합 완료! 전체 데이터 크기: {train.shape}")

🔄 월별 파일 병합 시작 (7월~12월)...
   - credit_preprocessing_month_7.parquet 로드 완료
   - credit_preprocessing_month_8.parquet 로드 완료
   - credit_preprocessing_month_9.parquet 로드 완료
   - credit_preprocessing_month_10.parquet 로드 완료
   - credit_preprocessing_month_11.parquet 로드 완료
   - credit_preprocessing_month_12.parquet 로드 완료
✅ 병합 완료! 전체 데이터 크기: (2400000, 858)


In [5]:
import pandas as pd
import os
import gc

# =========================================================
# ✅ Step 1-2: Test 데이터 로드 및 병합 (7~12월)
# =========================================================
FILE_PATH = r"C:\Users\CY2\github\DataScience\Credit Card"
months = range(7, 13)
test_list = []

print("🔄 Test 데이터(7~12월) 로드 및 병합 시작...")

for m in months:
    # 사용자님이 만드신 파일명 규칙 적용
    file_name = f"credit_test_month_{m}.parquet"
    full_path = os.path.join(FILE_PATH, file_name)
    
    if os.path.exists(full_path):
        # fastparquet 엔진 사용 (속도 및 호환성)
        try:
            df = pd.read_parquet(full_path, engine='fastparquet')
        except:
            df = pd.read_parquet(full_path) # 엔진 지정 없이 재시도
            
        test_list.append(df)
        print(f"   - {file_name} 로드 완료")
    else:
        print(f"   ⚠️ 파일 없음: {file_name} (경로를 확인해주세요)")

# 리스트가 비어있지 않다면 병합 진행
if test_list:
    # 1. 하나로 병합
    test = pd.concat(test_list, ignore_index=True)
    
    # 2. 메모리 정리 (리스트 삭제)
    del test_list
    gc.collect()
    
    print(f"\n✅ Test 데이터 병합 완료!")
    print(f"   - 데이터 크기(Shape): {test.shape}")
    print(f"   - 메모리 사용량: {test.memory_usage().sum() / 1024**2:.2f} MB")
    
    # (선택사항) 병합된 파일 저장해두기 - 필요 없으면 주석 처리
    # test.to_parquet(os.path.join(FILE_PATH, "test_merged_7_12.parquet"), index=False)
    
else:
    print("❌ 로드된 파일이 없습니다. 경로와 파일명을 다시 확인해주세요.")

🔄 Test 데이터(7~12월) 로드 및 병합 시작...
   - credit_test_month_7.parquet 로드 완료
   - credit_test_month_8.parquet 로드 완료
   - credit_test_month_9.parquet 로드 완료
   - credit_test_month_10.parquet 로드 완료
   - credit_test_month_11.parquet 로드 완료
   - credit_test_month_12.parquet 로드 완료

✅ Test 데이터 병합 완료!
   - 데이터 크기(Shape): (600000, 857)
   - 메모리 사용량: 1126.67 MB


In [6]:
import time
import gc
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from scipy.stats.mstats import winsorize
import warnings

warnings.filterwarnings('ignore')

# =========================================================
# ✅ Step 2: 전처리 (5등 분석가 로직 적용)
# =========================================================
print("\n🔄 Step 2: 전처리 (인코딩, 결측치, 이상치 처리) 시작...")
start_time_step2 = time.time()

# 1. 타겟 컬럼 및 ID 컬럼 설정 (데이터에 맞게 수정 필요)
TARGET_COLUMN = 'Segment'  # 혹은 'N_Category'
ID_COLUMN = 'ID'

# 2. 타겟 변수 인코딩 (LabelEncoder)
print("   [1/4] 타겟 변수 인코딩 중...")
if TARGET_COLUMN in train.columns:
    y = train[TARGET_COLUMN].copy()
    le_target = LabelEncoder()
    # 안전하게 문자열로 변환 후 인코딩 (숫자/문자 혼용 방지)
    y_encoded = le_target.fit_transform(y.astype(str))
    num_classes = len(le_target.classes_)
    print(f"      - 타겟 클래스 개수: {num_classes}개 ({le_target.classes_})")
else:
    raise ValueError(f"❌ Error: Train 데이터에 '{TARGET_COL}' 컬럼이 없습니다.")

# 3. 데이터 분리 및 메모리 확보
print("   [2/4] 특징(X)과 정답(y) 분리 및 메모리 정리...")
# Train에서 불필요한 컬럼 제거
X = train.drop(columns=[ID_COLUMN, TARGET_COLUMN] if ID_COLUMN in train.columns else [TARGET_COLUMN])

# Test에서 ID 분리 및 불필요 컬럼 제거
if ID_COLUMN in test.columns:
    test_ids = test[ID_COLUMN].copy()
    X_test = test.drop(columns=[ID_COLUMN])
else:
    test_ids = test.index # ID 컬럼 없으면 인덱스 사용
    X_test = test.copy()

# ⚠️ 핵심: 원본 데이터 삭제로 메모리 확보 (8GB 환경 필수)
del train, test, y
gc.collect()

# 수치형/범주형 컬럼 자동 분류
numerical_features = X.select_dtypes(include=np.number).columns.tolist()
categorical_features = X.select_dtypes(exclude=np.number).columns.tolist()
print(f"      - 수치형 변수: {len(numerical_features)}개")
print(f"      - 범주형 변수: {len(categorical_features)}개")




🔄 Step 2: 전처리 (인코딩, 결측치, 이상치 처리) 시작...
   [1/4] 타겟 변수 인코딩 중...
      - 타겟 클래스 개수: 5개 (['A' 'B' 'C' 'D' 'E'])
   [2/4] 특징(X)과 정답(y) 분리 및 메모리 정리...
      - 수치형 변수: 808개
      - 범주형 변수: 48개


In [78]:
# # 4. 결측치 처리 (SimpleImputer)
# print("   [3/4] 결측치 처리 (Median & Most Frequent)...")

# # 수치형: 중앙값(Median)
# if numerical_features:
#     num_imputer = SimpleImputer(strategy='median')
#     X[numerical_features] = num_imputer.fit_transform(X[numerical_features])
#     # Test 데이터는 Train의 중앙값 기준으로 채움 (Data Leakage 방지)
#     X_test[numerical_features] = num_imputer.transform(X_test[numerical_features])

# # 범주형: 최빈값(Most Frequent)
# if categorical_features:
#     cat_imputer = SimpleImputer(strategy='most_frequent')
#     X[categorical_features] = cat_imputer.fit_transform(X[categorical_features])
#     X_test[categorical_features] = cat_imputer.transform(X_test[categorical_features])

In [ ]:
# # 메모리 터지면 이거써!
# # 오래걸려도 메모리는 안터지는방법

# import numpy as np

# print("   [3/4] 메모리 절약형 결측치 처리 시작...")

# # 1. 수치형 변수: 중앙값(Median)으로 채우기
# if numerical_features:
#     print(f"      - 수치형 변수 {len(numerical_features)}개 처리 중...")
#     for col in numerical_features:
#         # Train 데이터의 중앙값 계산
#         median_val = X[col].median()
#         # Train/Test 각각 채우기 (inplace=True와 유사한 방식으로 메모리 절약)
#         X[col] = X[col].fillna(median_val).astype('float32')
#         X_test[col] = X_test[col].fillna(median_val).astype('float32')
#     gc.collect() # 중간 메모리 정리

# # 2. 범주형 변수: 최빈값(Most Frequent)으로 채우기
# if categorical_features:
#     print(f"      - 범주형 변수 {len(categorical_features)}개 처리 중...")
#     for col in categorical_features:
#         # 최빈값이 없을 경우를 대비해 예외 처리
#         mode_series = X[col].mode()
#         if not mode_series.empty:
#             mode_val = mode_series[0]
#             X[col] = X[col].fillna(mode_val)
#             X_test[col] = X_test[col].fillna(mode_val)
#     gc.collect()

# print("   ✅ 결측치 처리 및 float32 형변환 완료.")

   [3/4] 메모리 절약형 결측치 처리 시작...
      - 수치형 변수 808개 처리 중...


KeyboardInterrupt: 

In [7]:
# 결측치 처리 느릴떄 

import numpy as np

print("   [3/4] 초고속+저용량 결측치 처리 시작...")

if numerical_features:
    # 1. 중앙값을 한꺼번에 계산 (이건 메모리를 생각보다 많이 안 먹습니다)
    print("      - 수치형 변수 중앙값 계산 중...")
    medians = X[numerical_features].median() 
    
    # 2. 채우기 작업만 루프로 진행 (이미 계산된 값을 쓰므로 훨씬 빠름)
    print(f"      - {len(numerical_features)}개 컬럼 결측치 채우기 시작...")
    for col in numerical_features:
        X[col] = X[col].fillna(medians[col]).astype('float32')
        X_test[col] = X_test[col].fillna(medians[col]).astype('float32')
    gc.collect()

# 범주형도 동일하게 최빈값 미리 계산
if categorical_features:
    print("      - 범주형 변수 최빈값 계산 중...")
    modes = X[categorical_features].mode().iloc[0]
    
    for col in categorical_features:
        X[col] = X[col].fillna(modes[col])
        X_test[col] = X_test[col].fillna(modes[col])
    gc.collect()

print("   ✅ 최적화 결측치 처리 완료.")

   [3/4] 초고속+저용량 결측치 처리 시작...
      - 수치형 변수 중앙값 계산 중...
      - 808개 컬럼 결측치 채우기 시작...
      - 범주형 변수 최빈값 계산 중...
   ✅ 최적화 결측치 처리 완료.


In [8]:
# 5. 이상치 처리 (Winsorization & Clipping) - 5등 분석가 핵심
print("   [4/4] 이상치 처리 (Winsorization 상하위 1%)...")
winsorize_alpha = 0.01 # 상하위 1%

for col in numerical_features:
    # Train: 상하위 1% 값을 경계값으로 변환 (Winsorization)
    X[col] = winsorize(X[col], limits=(winsorize_alpha, winsorize_alpha))
    
    # Test: Train에서 계산된 경계값(1%, 99%)을 기준으로 자름 (Clipping)
    # ⚠️ Test 데이터에 직접 winsorize를 쓰면 안 됨 (기준이 달라짐)
    q_low = np.percentile(X[col], winsorize_alpha * 100)
    q_high = np.percentile(X[col], (1 - winsorize_alpha) * 100)
    X_test[col] = np.clip(X_test[col], q_low, q_high)

end_time_step2 = time.time()
print(f"✅ Step 2 완료. (소요 시간: {end_time_step2 - start_time_step2:.2f} 초)")
print(f"   - 최종 학습 데이터 크기: {X.shape}")
print(f"   - 최종 테스트 데이터 크기: {X_test.shape}")

   [4/4] 이상치 처리 (Winsorization 상하위 1%)...
✅ Step 2 완료. (소요 시간: 123.26 초)
   - 최종 학습 데이터 크기: (2400000, 856)
   - 최종 테스트 데이터 크기: (600000, 856)


In [9]:
import numpy as np
import pandas as pd
import gc

def remove_zero_variance_features(X, X_test, numerical_features):
    print("🔄 분산이 0인 변수(모든 값이 똑같은 컬럼) 탐색 및 제거 시작...")
    
    # 1. 제거할 컬럼 찾기 (For문으로 가볍게 검사)
    cols_to_drop = []
    
    # 전체를 한 번에 계산하지 않고 하나씩 확인해서 메모리 보호
    for col in numerical_features:
        if col in X.columns:
            # 최소값과 최대값이 같으면 -> 모든 값이 똑같다는 뜻 (분산 0)
            # (결측치가 이미 처리된 상태라고 가정)
            if X[col].min() == X[col].max():
                cols_to_drop.append(col)
    
    # 2. 실제 제거 작업
    if cols_to_drop:
        print(f"   🚨 총 {len(cols_to_drop)}개의 의미 없는 변수(상수)를 발견했습니다.")
        print(f"      예시: {cols_to_drop[:5]} ...")
        
        # inplace=True로 메모리 복사 없이 즉시 삭제
        X.drop(columns=cols_to_drop, inplace=True)
        # Test 데이터에서도 똑같이 삭제
        X_test.drop(columns=cols_to_drop, inplace=True)
        
        # 관리 중인 리스트에서도 삭제
        numerical_features = [c for c in numerical_features if c not in cols_to_drop]
        print("   ✅ 제거 완료.")
    else:
        print("   ✅ 제거할 변수가 없습니다. (모든 변수가 변화량이 있음)")
        
    print(f"   -> 현재 남은 전체 피처 수: {X.shape[1]}개")
    
    # 메모리 청소
    gc.collect()
    
    return X, X_test, numerical_features

# ==========================================
# 실행
# ==========================================
X, X_test, numerical_features = remove_zero_variance_features(X, X_test, numerical_features)

🔄 분산이 0인 변수(모든 값이 똑같은 컬럼) 탐색 및 제거 시작...
   🚨 총 226개의 의미 없는 변수(상수)를 발견했습니다.
      예시: ['유효카드수_체크_가족', '이용가능카드수_체크_가족', '이용카드수_체크_가족', '이용금액_R3M_체크_가족', '연회비할인카드수_B0M'] ...
   ✅ 제거 완료.
   -> 현재 남은 전체 피처 수: 630개


In [10]:
import numpy as np
import pandas as pd
import gc

def apply_log_transformation(X, X_test, numerical_features):
    print("🔄 로그 변환 (Log Transformation) 시작...")
    
    # 1. 로그 변환 대상 컬럼 찾기
    # 금액, 원금, 평잔 등의 키워드가 포함된 컬럼을 찾습니다.
    # 이미 '_log1p'가 붙은 컬럼은 제외합니다 (중복 방지).
    log_keywords = ['금액', '원금', '평잔', 'Amount', 'Balance']
    target_cols = [c for c in numerical_features if any(k in c for k in log_keywords) and '_log1p' not in c]
    
    print(f"   -> 대상 컬럼: {len(target_cols)}개 발견")
    
    added_features = []
    
    # 2. 변환 수행 (한 번에 하나씩 처리하여 메모리 보호)
    for col in target_cols:
        if col in X.columns:
            new_col_name = f"{col}_log1p"
            
            # (1) 학습 데이터(X) 변환
            # 음수가 있는 경우 최솟값을 빼서 양수로 만든 뒤 로그 적용
            min_val = X[col].min()
            if min_val < 0:
                X[new_col_name] = np.log1p(X[col] - min_val).astype('float32')
                # 테스트 데이터에도 똑같은 min_val을 적용해야 함 (기준 통일)
                if col in X_test.columns:
                    X_test[new_col_name] = np.log1p(X_test[col] - min_val).astype('float32')
            else:
                X[new_col_name] = np.log1p(X[col]).astype('float32')
                if col in X_test.columns:
                    X_test[new_col_name] = np.log1p(X_test[col]).astype('float32')
            
            added_features.append(new_col_name)

    # 3. 새로운 컬럼을 리스트에 추가
    numerical_features.extend(added_features)
    
    print(f"   ✅ {len(added_features)}개 변수에 대해 로그 변환 완료!")
    print(f"   -> 현재 전체 피처 수: {X.shape[1]}개")
    
    # 메모리 청소
    gc.collect()
    
    return X, X_test, numerical_features

# ==========================================
# 실행
# ==========================================
X, X_test, numerical_features = apply_log_transformation(X, X_test, numerical_features)

🔄 로그 변환 (Log Transformation) 시작...
   -> 대상 컬럼: 208개 발견
   ✅ 208개 변수에 대해 로그 변환 완료!
   -> 현재 전체 피처 수: 838개


In [11]:
import numpy as np
import pandas as pd
import gc

def create_avg_transaction_feature(X, X_test, numerical_features):
    print("🔄 평균 거래 금액 (Average Transaction Amount) 생성 중...")
    
    epsilon = 1e-6 # 0으로 나누기 방지용 아주 작은 수
    
    # 1. '총 이용금액'과 '총 이용건수' 컬럼 준비
    # (이미 만들어져 있다면 가져다 쓰고, 없다면 지금 계산합니다)
    
    # [A] 총 이용금액 (R12M: 최근 1년)
    if '총이용금액_R12M' not in X.columns:
        amt_cols = [c for c in X.columns if '이용금액' in c and 'R12M' in c and '_log1p' not in c]
        if amt_cols:
            print(f"   -> '총이용금액_R12M' 생성 (대상 컬럼 {len(amt_cols)}개 합산)")
            X['총이용금액_R12M'] = X[amt_cols].sum(axis=1).astype('float32')
            if '총이용금액_R12M' not in numerical_features: numerical_features.append('총이용금액_R12M')
            
            # Test 데이터도 동일하게
            if set(amt_cols).issubset(X_test.columns):
                X_test['총이용금액_R12M'] = X_test[amt_cols].sum(axis=1).astype('float32')

    # [B] 총 이용건수 (R12M: 최근 1년)
    if '총이용건수_R12M' not in X.columns:
        cnt_cols = [c for c in X.columns if '이용건수' in c and 'R12M' in c]
        if cnt_cols:
            print(f"   -> '총이용건수_R12M' 생성 (대상 컬럼 {len(cnt_cols)}개 합산)")
            X['총이용건수_R12M'] = X[cnt_cols].sum(axis=1).astype('float32')
            if '총이용건수_R12M' not in numerical_features: numerical_features.append('총이용건수_R12M')
            
            # Test 데이터도 동일하게
            if set(cnt_cols).issubset(X_test.columns):
                X_test['총이용건수_R12M'] = X_test[cnt_cols].sum(axis=1).astype('float32')

    # 2. 평균 거래 금액 계산 (금액 / 건수)
    if '총이용금액_R12M' in X.columns and '총이용건수_R12M' in X.columns:
        new_col = '평균거래금액_R12M'
        
        # Train 계산
        # (건수에 epsilon을 더해서 0으로 나누는 것을 방지)
        X[new_col] = (X['총이용금액_R12M'] / (X['총이용건수_R12M'] + epsilon)).astype('float32')
        
        # Test 계산
        if '총이용금액_R12M' in X_test.columns and '총이용건수_R12M' in X_test.columns:
            X_test[new_col] = (X_test['총이용금액_R12M'] / (X_test['총이용건수_R12M'] + epsilon)).astype('float32')
            
        numerical_features.append(new_col)
        print(f"   ✅ '{new_col}' 생성 완료!")
    else:
        print("   ⚠️ 주의: '이용금액'이나 '이용건수' 관련 컬럼이 없어서 평균금액을 만들지 못했습니다.")

    print(f"   -> 현재 전체 피처 수: {X.shape[1]}개")
    gc.collect()
    
    return X, X_test, numerical_features

# ==========================================
# 실행
# ==========================================
X, X_test, numerical_features = create_avg_transaction_feature(X, X_test, numerical_features)


🔄 평균 거래 금액 (Average Transaction Amount) 생성 중...
   -> '총이용금액_R12M' 생성 (대상 컬럼 16개 합산)
   -> '총이용건수_R12M' 생성 (대상 컬럼 9개 합산)
   ✅ '평균거래금액_R12M' 생성 완료!
   -> 현재 전체 피처 수: 841개


In [12]:
import numpy as np
import pandas as pd
import gc

def create_pattern_and_risk_features(X, X_test, numerical_features, categorical_features):
    print("🔄 소비 패턴 및 신용 리스크 파생변수 생성 중...")
    
    epsilon = 1e-6 # 나눗셈 에러 방지용
    
    # ---------------------------------------------------------
    # 1. 소비 패턴 (Consumption Pattern)
    # ---------------------------------------------------------
    
    # [A] 온라인 소비 비중 (온라인 금액 / 총 금액)
    # '온라인'과 '금액'이 들어간 컬럼 찾기
    online_cols = [c for c in X.columns if '온라인' in c and '금액' in c and '_log1p' not in c]
    
    if online_cols and '총이용금액_R12M' in X.columns:
        print(f"   -> [패턴] 온라인 관련 컬럼 {len(online_cols)}개 발견. 비중 계산 중...")
        
        # 온라인 금액 합계 (float32)
        online_sum = X[online_cols].sum(axis=1).astype('float32')
        online_sum_test = X_test[online_cols].sum(axis=1).astype('float32')
        
        # 비중 계산
        X['온라인소비비중'] = (online_sum / (X['총이용금액_R12M'] + epsilon)).astype('float32')
        X_test['온라인소비비중'] = (online_sum_test / (X_test['총이용금액_R12M'] + epsilon)).astype('float32')
        
        numerical_features.append('온라인소비비중')
    
    # [B] 최근 소비 집중도 ((R3M * 4) / R12M)
    # 최근 3개월(R3M) 이용금액 찾기
    r3m_cols = [c for c in X.columns if '이용금액' in c and 'R3M' in c and '_log1p' not in c]
    
    if r3m_cols and '총이용금액_R12M' in X.columns:
        print(f"   -> [패턴] 최근 3개월(R3M) 데이터 발견. 소비 집중도 계산 중...")
        
        r3m_sum = X[r3m_cols].sum(axis=1).astype('float32')
        r3m_sum_test = X_test[r3m_cols].sum(axis=1).astype('float32')
        
        # 연간 환산(x4) 후 비율 계산
        X['최근소비집중도'] = ((r3m_sum * 4) / (X['총이용금액_R12M'] + epsilon)).astype('float32')
        X_test['최근소비집중도'] = ((r3m_sum_test * 4) / (X_test['총이용금액_R12M'] + epsilon)).astype('float32')
        
        numerical_features.append('최근소비집중도')

    # ---------------------------------------------------------
    # 2. 신용도 및 위험 관리 (Risk Management)
    # ---------------------------------------------------------

    # [C] 한도 소진율 (청구금액 / 한도금액)
    limit_cols = [c for c in X.columns if '한도' in c and '금액' in c]
    bill_cols = [c for c in X.columns if '청구' in c and '금액' in c]
    
    if limit_cols and bill_cols:
        print(f"   -> [위험] 한도 및 청구 데이터 발견. 한도소진율 계산 중...")
        
        # 가장 대표적인 컬럼 하나씩만 사용 (보통 0번 인덱스가 총한도/총청구)
        limit_val = X[limit_cols[0]]
        bill_val = X[bill_cols[0]]
        
        limit_val_test = X_test[limit_cols[0]]
        bill_val_test = X_test[bill_cols[0]]
        
        # 비율 계산 및 이상치 제어 (1.5배 초과 시 1.5로 고정)
        X['한도소진율'] = (bill_val / (limit_val + epsilon)).clip(0, 1.5).astype('float32')
        X_test['한도소진율'] = (bill_val_test / (limit_val_test + epsilon)).clip(0, 1.5).astype('float32')
        
        numerical_features.append('한도소진율')

    # [D] 연체 경험 유무 (연체 컬럼 합계 > 0 ?)
    delinq_cols = [c for c in X.columns if '연체' in c]
    
    if delinq_cols:
        print(f"   -> [위험] 연체 관련 컬럼 {len(delinq_cols)}개 발견. 연체 경험 플래그 생성...")
        
        # 연체 금액이나 횟수가 하나라도 있으면 1, 아니면 0
        X['연체경험유무'] = (X[delinq_cols].sum(axis=1) > 0).astype('int8')
        X_test['연체경험유무'] = (X_test[delinq_cols].sum(axis=1) > 0).astype('int8')
        
        if '연체경험유무' not in categorical_features:
            categorical_features.append('연체경험유무')

    print(f"   ✅ 생성 완료! 현재 전체 피처 수: {X.shape[1]}개")
    gc.collect()
    
    return X, X_test, numerical_features, categorical_features

# ==========================================
# 실행
# ==========================================
X, X_test, numerical_features, categorical_features = create_pattern_and_risk_features(X, X_test, numerical_features, categorical_features)

🔄 소비 패턴 및 신용 리스크 파생변수 생성 중...
   -> [패턴] 온라인 관련 컬럼 7개 발견. 비중 계산 중...
   -> [패턴] 최근 3개월(R3M) 데이터 발견. 소비 집중도 계산 중...
   -> [위험] 한도 및 청구 데이터 발견. 한도소진율 계산 중...
   -> [위험] 연체 관련 컬럼 22개 발견. 연체 경험 플래그 생성...
   ✅ 생성 완료! 현재 전체 피처 수: 845개


In [13]:
import numpy as np

def apply_2nd_place_limit_logic_fast(X, X_test):
    print("🚀 [초고속 모드] '카드이용한도금액_A수준복합' 변수 생성 중...")
    threshold = 170344 
    
    col_b1 = '카드이용한도금액_B1M'
    col_b2 = '카드이용한도금액_B2M'

    # .apply 대신 벡터 연산 사용 (True=1, False=0 임을 이용)
    # 두 조건이 모두 참이면 1+1=2, 하나만 참이면 1+0=1, 둘 다 아니면 0+0=0이 됩니다.
    X['카드이용한도금액_A수준복합'] = (X[col_b1] >= threshold).astype(int) + (X[col_b2] >= threshold).astype(int)
    X_test['카드이용한도금액_A수준복합'] = (X_test[col_b1] >= threshold).astype(int) + (X_test[col_b2] >= threshold).astype(int)
    
    # [공식 등록] 리스트에 추가 (중복 방지)
    if '카드이용한도금액_A수준복합' not in numerical_features:
        numerical_features.append('카드이용한도금액_A수준복합')
        
    print(f"   ✅ 1초 만에 생성 완료! (기준점: {threshold:,.0f}원)")
    return X, X_test

# 실행
X, X_test = apply_2nd_place_limit_logic_fast(X, X_test)

🚀 [초고속 모드] '카드이용한도금액_A수준복합' 변수 생성 중...
   ✅ 1초 만에 생성 완료! (기준점: 170,344원)


In [14]:
import numpy as np
import pandas as pd
import gc

print("🚀 2등 솔루션 알고리즘 적용: 핵심 파생변수 4종 자동 생성 중...")

# --------------------------------------------------------------------------------
# 🕵️‍♀️ [알고리즘 1] 스마트 컬럼 탐색기 (이름이 달라도 찾아냅니다)
# --------------------------------------------------------------------------------
def find_and_create_avg(df, keywords, term_suffix, new_col_name):
    """
    df: 데이터프레임
    keywords: 포함되어야 할 단어들 (예: ['CA', '해외'])
    term_suffix: 기간 접미사 패턴 (예: ['B1M', 'B2M', 'B3M'])
    new_col_name: 만들 변수 이름
    """
    # 1. 키워드가 모두 들어간 컬럼 후보 찾기
    candidates = [c for c in df.columns if all(k in c for k in keywords)]
    
    # 2. 기간(B1M, B2M...)별로 매칭되는 컬럼 찾기
    selected_cols = []
    found_suffixes = []
    
    for suffix in term_suffix:
        # 후보 중 해당 접미사(예: B1M)를 가진 놈 찾기
        match = [c for c in candidates if suffix in c]
        if match:
            selected_cols.append(match[0]) # 첫 번째 매칭된 것 선택
            found_suffixes.append(suffix)
            
    # 3. 평균 계산 (2등 스타일: 데이터가 하나라도 있으면 평균 냄)
    if selected_cols:
        print(f"   👉 [{new_col_name}] 생성완료! (사용된 원본: {selected_cols})")
        # fillna(0)으로 결측치 방어 후 평균 계산
        return df[selected_cols].fillna(0).mean(axis=1)
    else:
        print(f"   ⚠️ [{new_col_name}] 생성 실패: 관련 컬럼을 찾지 못했습니다. (키워드: {keywords})")
        return pd.Series(0, index=df.index)

# --------------------------------------------------------------------------------
# 1️⃣ 변수 생성 적용 (Train / Test 모두 적용)
# --------------------------------------------------------------------------------

# 이용금액_선결제_B0M (선결제 + 당월)
# 알고리즘: '선결제'와 'B0M'이 들어간 컬럼을 찾아 그대로 복사
prepay_col = [c for c in X.columns if '선결제' in c and 'B0M' in c]
if prepay_col:
    col_name = prepay_col[0]
    X['이용금액_선결제_B0M'] = X[col_name].fillna(0)
    X_test['이용금액_선결제_B0M'] = X_test[col_name].fillna(0)
    print(f"   👉 [이용금액_선결제_B0M] 매핑 완료! (원본: {col_name})")
else:
    X['이용금액_선결제_B0M'] = 0
    X_test['이용금액_선결제_B0M'] = 0
    print("   ⚠️ [이용금액_선결제_B0M] 관련 컬럼 없음 -> 0으로 채움")

#  정상청구원금_B5M
# '정상' + '청구' + 'B5M' 키워드
normal_bill_col = [c for c in X.columns if '정상' in c and '청구' in c and 'B5M' in c]
if normal_bill_col:
    X['정상청구원금_B5M'] = X[normal_bill_col[0]].fillna(0)
    X_test['정상청구원금_B5M'] = X_test[normal_bill_col[0]].fillna(0)
    print(f"   👉 [정상청구원금_B5M] 매핑 완료! (원본: {normal_bill_col[0]})")
else:
    X['정상청구원금_B5M'] = 0
    X_test['정상청구원금_B5M'] = 0

# --------------------------------------------------------------------------------
# 2️⃣ 리스트 등록 및 정리 (필수!)
# --------------------------------------------------------------------------------
new_features = ['평잔_CA_해외_3M', '이용금액_선결제_B0M', '할부금액_무이자_3M_R12M', '정상청구원금_B5M']

for col in new_features:
    if col not in numerical_features:
        numerical_features.append(col)

# 중복 제거
numerical_features = list(dict.fromkeys(numerical_features))
categorical_features = list(dict.fromkeys(categorical_features))

print(f"\n✅ 최종 완료. 현재 수치형 피처 수: {len(numerical_features)}개")
gc.collect()

print(f"   ✅ 생성 완료! 현재 전체 피처 수: {X.shape[1]}개")

🚀 2등 솔루션 알고리즘 적용: 핵심 파생변수 4종 자동 생성 중...
   👉 [이용금액_선결제_B0M] 매핑 완료! (원본: 이용횟수_선결제_B0M)
   👉 [정상청구원금_B5M] 매핑 완료! (원본: 정상청구원금_B5M)

✅ 최종 완료. 현재 수치형 피처 수: 798개
   ✅ 생성 완료! 현재 전체 피처 수: 846개


In [15]:
# 1. 중복된 이름표 싹 정리하기 (순서 유지하며 중복 제거)
numerical_features = list(dict.fromkeys(numerical_features))
categorical_features = list(dict.fromkeys(categorical_features))

print(f"✅ 중복 제거 완료! 현재 수치형 변수 개수: {len(numerical_features)}개")

# 2. 혹시 모르니 데이터(X)에 중복된 컬럼이 있는지도 확인 (있으면 제거)
X = X.loc[:, ~X.columns.duplicated()].copy()
X_test = X_test.loc[:, ~X_test.columns.duplicated()].copy()

print("✅ 데이터프레임 컬럼 중복 제거 완료.")

✅ 중복 제거 완료! 현재 수치형 변수 개수: 798개
✅ 데이터프레임 컬럼 중복 제거 완료.


In [16]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import QuantileTransformer
from sklearn.cluster import MiniBatchKMeans
import gc
import warnings

warnings.filterwarnings('ignore')

# =========================================================
# ✅ Step 3-6 & 7: 클러스터링 및 최종 정리 함수
# =========================================================
def apply_clustering_and_cleanup(df, numerical_features, categorical_features, fit_objects=None):
    print("🔄 (6) 클러스터링 및 (7) 최종 정리 시작...")
    
    # 1. 변수 초기화 및 설정
    X_eng = df.copy() # 원본 보존을 위해 복사
    current_num_features = numerical_features[:]
    current_cat_features = categorical_features[:]
    
    RANDOM_STATE = 42
    
    # 학습 모드 확인
    is_training = (fit_objects is None)
    if is_training: fit_objects = {}

    # ---------------------------------------------------------
    # (6) 클러스터링 (MiniBatchKMeans)
    # ---------------------------------------------------------
    # 로그 변환된 컬럼들만 찾기
    cluster_cols_log = [col for col in current_num_features if '_log1p' in col and col in X_eng.columns]
    
    # 피처가 2개 이상이어야 클러스터링 가능
    if len(cluster_cols_log) >= 2:
        if is_training:
            print(f"   -> 클러스터링 학습 중 (대상 변수 {len(cluster_cols_log)}개)...")
            
            # (A) 분포 변환 (QuantileTransformer)
            qt = QuantileTransformer(output_distribution='normal', random_state=RANDOM_STATE)
            
            # 메모리 보호: 10만 개만 샘플링해서 학습 (fit)
            sample_size = min(100000, len(X_eng))
            sample_idx = X_eng[cluster_cols_log].sample(n=sample_size, random_state=RANDOM_STATE).index
            
            # 샘플 데이터로 fit
            qt.fit(X_eng.loc[sample_idx, cluster_cols_log].fillna(0))
            
            # (B) 클러스터링 학습 (MiniBatchKMeans)
            kmeans = MiniBatchKMeans(n_clusters=5, random_state=RANDOM_STATE, batch_size=4096, n_init=3)
            
            # 전체 데이터 변환 (Transform)
            X_scaled = qt.transform(X_eng[cluster_cols_log].fillna(0))
            
            # 예측 및 int8 변환
            X_eng['cluster_feature'] = kmeans.fit_predict(X_scaled).astype('int8')
            
            # 객체 저장 (Test 적용을 위해)
            fit_objects['kmeans'] = kmeans
            fit_objects['qt_cluster'] = qt
            fit_objects['cluster_cols_log'] = cluster_cols_log
            
        else:
            # Test 모드: 저장된 객체가 있을 때만 실행
            if 'kmeans' in fit_objects and 'qt_cluster' in fit_objects:
                qt = fit_objects['qt_cluster']
                kmeans = fit_objects['kmeans']
                cols = fit_objects['cluster_cols_log']
                
                # Test 데이터에 해당 컬럼이 다 있는지 확인
                cols = [c for c in cols if c in X_eng.columns]
                
                if len(cols) > 0:
                    # 없는 컬럼은 0으로 채우는 안전장치 (필요시)
                    # 여기선 있는 것만 가지고 transform 시도 (차원 주의)
                    try:
                        X_scaled = qt.transform(X_eng[cols].fillna(0))
                        X_eng['cluster_feature'] = kmeans.predict(X_scaled).astype('int8')
                    except ValueError:
                        print("   ⚠️ 경고: Test 데이터 차원 불일치로 클러스터링 0 처리")
                        X_eng['cluster_feature'] = 0
                else:
                    X_eng['cluster_feature'] = 0
            else:
                X_eng['cluster_feature'] = 0
        
        # 범주형 변수 리스트에 추가
        if 'cluster_feature' not in current_cat_features: 
            current_cat_features.append('cluster_feature')

    # ---------------------------------------------------------
    # (7) 최종 정리 및 안정성 확인
    # ---------------------------------------------------------
    # 존재하는 컬럼만 남기고 정렬
    final_num = sorted(list(set([f for f in current_num_features if f in X_eng.columns])))
    final_cat = sorted(list(set([f for f in current_cat_features if f in X_eng.columns])))
    
    # 덮어쓰기 (메모리 정리)
    X_eng = X_eng[final_num + final_cat].copy()

    # NaN/Inf 처리 (수치형만)
    X_eng[final_num] = X_eng[final_num].replace([np.inf, -np.inf], 0).fillna(0).astype('float32')
    
    gc.collect()
    print(f"   ✅ 완료. 최종 컬럼 수: {X_eng.shape[1]}")
    
    return X_eng, final_num, final_cat, fit_objects

# =========================================================
# 🚀 실행 코드 (Train -> Test)
# =========================================================

# 1. Train 적용
# (X, numerical_features, categorical_features는 이미 정의되어 있어야 함)
X, numerical_features, categorical_features, fit_objects = apply_clustering_and_cleanup(
    X, numerical_features, categorical_features, fit_objects=None
)

# 2. Test 적용 (fit_objects 전달 필수)
X_test, _, _, _ = apply_clustering_and_cleanup(
    X_test, numerical_features, categorical_features, fit_objects=fit_objects
)

🔄 (6) 클러스터링 및 (7) 최종 정리 시작...
   -> 클러스터링 학습 중 (대상 변수 208개)...


Exception in thread Thread-4 (_readerthread):
Traceback (most recent call last):
  File "c:\Users\CY2\miniconda3\envs\DS\Lib\threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "c:\Users\CY2\miniconda3\envs\DS\Lib\threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\CY2\miniconda3\envs\DS\Lib\subprocess.py", line 1599, in _readerthread
    buffer.append(fh.read())
                  ^^^^^^^^^
  File "<frozen codecs>", line 322, in decode
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc0 in position 4: invalid start byte


   ✅ 완료. 최종 컬럼 수: 847
🔄 (6) 클러스터링 및 (7) 최종 정리 시작...
   ✅ 완료. 최종 컬럼 수: 847


In [17]:
import gc

print("🚀 클러스터링 및 최종 정리 함수 적용 중...")

# 1. Train 데이터 적용 (fit_objects 생성)
X, numerical_features, categorical_features, fit_objects = apply_clustering_and_cleanup(
    X, numerical_features, categorical_features, fit_objects=None
)

# 2. Test 데이터 적용 (생성된 fit_objects 사용)
X_test, _, _, _ = apply_clustering_and_cleanup(
    X_test, numerical_features, categorical_features, fit_objects=fit_objects
)

# 3. 메모리 정리
gc.collect()

print("\n✨ 전처리 완료! 데이터가 최종 확정되었습니다.")
print(f"   - 최종 Train 크기: {X.shape}")
print(f"   - 최종 Test 크기: {X_test.shape}")

🚀 클러스터링 및 최종 정리 함수 적용 중...
🔄 (6) 클러스터링 및 (7) 최종 정리 시작...
   -> 클러스터링 학습 중 (대상 변수 208개)...
   ✅ 완료. 최종 컬럼 수: 847
🔄 (6) 클러스터링 및 (7) 최종 정리 시작...
   ✅ 완료. 최종 컬럼 수: 847

✨ 전처리 완료! 데이터가 최종 확정되었습니다.
   - 최종 Train 크기: (2400000, 847)
   - 최종 Test 크기: (600000, 847)


In [19]:
import time
import gc
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb

print("\n🔄 Step 5: 피처 선택(Feature Selection) 시작... (원본 보존 모드)")
start_time_step5 = time.time()

try:
    # 1. 피처 선택을 위한 임시 데이터 생성 (원본 X 복사)
    X_temp_fs = X.copy()
    
    # 2. 범주형 변수 임시 라벨 인코딩
    cat_features_for_fs = X_temp_fs.select_dtypes(include=['object', 'category']).columns.tolist()
    for col in cat_features_for_fs:
        le_fs = LabelEncoder()
        X_temp_fs[col] = le_fs.fit_transform(X_temp_fs[col].astype(str))
    
    print(f"   - 피처 선택용 임시 인코딩 완료 ({len(cat_features_for_fs)}개).")

    # 3. LightGBM을 이용한 피처 중요도 계산
    lgbm_selector = lgb.LGBMClassifier(
        n_estimators=100, random_state=42, objective='multiclass', 
        metric='multi_logloss', n_jobs=-1, force_col_wise=True
    )
    lgbm_selector.fit(X_temp_fs, y_encoded)

    # 4. 중요도 상위 400개 피처 선택 (중앙값 대신 안정적인 개수 선택)
    importances = lgbm_selector.feature_importances_
    feat_imp_df = pd.DataFrame({'feature': X_temp_fs.columns, 'importance': importances})
    feat_imp_df = feat_imp_df.sort_values(by='importance', ascending=False)

    selected_feature_names = feat_imp_df.head(400)['feature'].tolist()

    print(f"   - 피처 선택 완료. (850개 내외 -> {len(selected_feature_names)}개 추출)")

    # 5. ⭐ [핵심 변경] 원본 X, X_test는 건드리지 않고 '학습 전용 변수' 생성
    X_train_fs = X[selected_feature_names].copy()
    X_test_fs = X_test[selected_feature_names].copy()

    # 학습용 리스트 업데이트 (원본 리스트와 별개로 관리)
    num_fs = X_train_fs.select_dtypes(include=np.number).columns.tolist()
    cat_fs = X_train_fs.select_dtypes(exclude=np.number).columns.tolist()
    
    print(f"   - ✅ 학습용 데이터 준비 완료: 수치형 {len(num_fs)}개, 범주형 {len(cat_fs)}개")
    print("   ⚠️ 주의: 아래 학습 셀(Optuna 등)에서 X 대신 'X_train_fs'를 사용하세요!")

    # 6. 메모리 정리
    del X_temp_fs, lgbm_selector, feat_imp_df
    gc.collect()
    
    end_time_step5 = time.time()
    print(f"✅ Step 5 완료. (소요 시간: {end_time_step5 - start_time_step5:.2f} 초)")

except Exception as e:
    print(f"❌ Step 5 실행 중 오류 발생: {e}")


🔄 Step 5: 피처 선택(Feature Selection) 시작... (원본 보존 모드)
   - 피처 선택용 임시 인코딩 완료 (48개).
❌ Step 5 실행 중 오류 발생: Unable to allocate 15.1 GiB for an array with shape (2400000, 847) and data type float64


In [20]:
import pandas as pd
import numpy as np
import xgboost as xgb
import category_encoders as ce
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import f1_score
import gc
import time
import optuna

# =============================================================================
# 🎛️ [컨트롤 타워] 여기만 바꾸면 알아서 변신합니다!
# =============================================================================
MODE = 'FEATURE_TEST'  # 👉 'FEATURE_TEST' 또는 'PARAM_TUNING' 중 선택

if MODE == 'FEATURE_TEST':
    print("🧪 [모드] 파생변수 실험 모드 (5% 샘플링)")
    SAMPLE_FRAC = 0.05    # 5%만 사용 (속도 극대화)
    N_TRIALS = 5          # 5번만 대충 돌려보고 점수 오르는지 확인
    N_FOLDS = 3           # Fold도 3개만 (빠르게)
    
elif MODE == 'PARAM_TUNING':
    print("🎚️ [모드] 파라미터 정밀 튜닝 모드 (20% 샘플링)")
    SAMPLE_FRAC = 0.2     # 20% 사용 (신뢰도 확보)
    N_TRIALS = 50         # 50번 돌려서 진짜 최적값 찾기
    N_FOLDS = 5           # Fold 5개 (국룰)

# =============================================================================

def objective(trial):
    # --- 1. 데이터 샘플링 ---
    # 전역 변수 SAMPLE_FRAC을 사용
    try:
        # X는 사용자님 코드에서 정의된 전체 데이터셋 변수명
        _, X_sample, _, y_sample = train_test_split(
            X_train_fs, y_encoded, test_size=SAMPLE_FRAC,
            random_state=42 + trial.number, stratify=y_encoded
        )
        X_sample = X_sample.reset_index(drop=True)
        # y_sample은 numpy array일 수 있으므로 hasattr 체크
        if hasattr(y_sample, 'reset_index'):
            y_sample = y_sample.reset_index(drop=True)
            
    except Exception as e:
        print(f"❌ Sampling Error: {e}")
        return 0.0
    
    gc.collect()

    # --- 2. 파라미터 탐색 범위 ---
    params = {
        'objective': 'multi:softprob',
        'num_class': num_classes, # 미리 정의되어 있어야 함
        'eval_metric': 'mlogloss',
        'tree_method': 'hist',
        'device': 'cuda',         # GPU 사용
        'random_state': 42,
        'nthread': -1,
        
        # 튜닝할 하이퍼파라미터
        'eta': trial.suggest_float('eta', 0.01, 0.1, log=True),
        'max_depth': trial.suggest_int('max_depth', 4, 10),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_float('gamma', 0.0, 5.0),
        'lambda': trial.suggest_float('lambda', 0.1, 100.0, log=True),
        'alpha': trial.suggest_float('alpha', 0.0, 10.0),
    }

    # --- 3. 학습 및 검증 ---
    oof_preds_proba = np.zeros((len(X_sample), num_classes))
    oof_labels = np.zeros(len(X_sample))
    
    kf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=42)
    cat_cols = X_sample.select_dtypes(include=['object', 'category']).columns.tolist()

    start_trial = time.time()
    
    for fold, (train_idx, val_idx) in enumerate(kf.split(X_sample, y_sample)):
        X_train_fold, X_val_fold = X_sample.iloc[train_idx].copy(), X_sample.iloc[val_idx].copy()
        y_train_fold, y_val_fold = y_sample[train_idx], y_sample[val_idx]

        # (1) Target Encoding
        if cat_cols:
            try:
                te = ce.TargetEncoder(cols=cat_cols, handle_missing='value', handle_unknown='value', smoothing=1.0)
                X_train_fold = te.fit_transform(X_train_fold, y_train_fold)
                X_val_fold = te.transform(X_val_fold)
            except: pass

        # (2) SMOTE
        try:
            smote = SMOTE(random_state=42 + fold, k_neighbors=5)
            X_train_resampled, y_train_resampled = smote.fit_resample(X_train_fold, y_train_fold)
        except:
            X_train_resampled, y_train_resampled = X_train_fold, y_train_fold

        X_train_resampled = X_train_resampled.astype(np.float32)
        X_val_fold = X_val_fold.astype(np.float32)

        # (3) XGBoost Train
        dtrain = xgb.DMatrix(X_train_resampled, label=y_train_resampled)
        dval = xgb.DMatrix(X_val_fold, label=y_val_fold)
        
        # Feature Test 모드일 때는 학습 라운드를 줄여서 속도 향상
        boost_rounds = 200 if MODE == 'FEATURE_TEST' else 500
        
        booster = xgb.train(
            params=params, dtrain=dtrain, num_boost_round=boost_rounds,
            evals=[(dval, 'eval')], early_stopping_rounds=30,
            verbose_eval=False
        )

        val_preds = booster.predict(dval, iteration_range=(0, booster.best_iteration))
        oof_preds_proba[val_idx] = val_preds
        oof_labels[val_idx] = y_val_fold
        
        del X_train_fold, X_val_fold, dtrain, dval, booster; gc.collect()

    # --- 4. 결과 집계 ---
    final_preds = np.argmax(oof_preds_proba, axis=1)
    macro_f1 = f1_score(oof_labels, final_preds, average='macro', zero_division=0)
    
    elapsed = time.time() - start_trial
    print(f"   [Trial {trial.number}] F1 Score: {macro_f1:.4f} (Time: {elapsed:.1f}s)")
    
    return macro_f1

# 🏆 실행
print(f"🚀 Optuna 시작! (Mode: {MODE}, Sample: {SAMPLE_FRAC*100}%)")
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=N_TRIALS)

print("\n🏆 [결과] 최적 파라미터:", study.best_params)
print("🏆 [결과] 최고 점수:", study.best_value)

🧪 [모드] 파생변수 실험 모드 (5% 샘플링)
🚀 Optuna 시작! (Mode: FEATURE_TEST, Sample: 5.0%)


[I 2026-02-06 17:47:16,296] A new study created in memory with name: no-name-10c5ec15-e0bf-42d1-a23a-f1fe5db0120d
[I 2026-02-06 17:47:16,337] Trial 0 finished with value: 0.0 and parameters: {}. Best is trial 0 with value: 0.0.
[I 2026-02-06 17:47:16,339] Trial 1 finished with value: 0.0 and parameters: {}. Best is trial 0 with value: 0.0.
[I 2026-02-06 17:47:16,345] Trial 2 finished with value: 0.0 and parameters: {}. Best is trial 0 with value: 0.0.
[I 2026-02-06 17:47:16,366] Trial 3 finished with value: 0.0 and parameters: {}. Best is trial 0 with value: 0.0.
[I 2026-02-06 17:47:16,384] Trial 4 finished with value: 0.0 and parameters: {}. Best is trial 0 with value: 0.0.


❌ Sampling Error: name 'X_train_fs' is not defined
❌ Sampling Error: name 'X_train_fs' is not defined
❌ Sampling Error: name 'X_train_fs' is not defined
❌ Sampling Error: name 'X_train_fs' is not defined
❌ Sampling Error: name 'X_train_fs' is not defined

🏆 [결과] 최적 파라미터: {}
🏆 [결과] 최고 점수: 0.0


In [ ]:
# import pandas as pd
# import numpy as np
# import xgboost as xgb
# import category_encoders as ce
# from imblearn.over_sampling import SMOTE
# from sklearn.model_selection import StratifiedKFold, train_test_split
# from sklearn.metrics import f1_score
# import gc
# import time

# def objective(trial):
#     # --- 1. 설정 ---
#     SAMPLE_FRAC = 0.05  # ⚡ 속도 확인을 위해 5%로 줄여서 먼저 테스트해보세요!
#     N_FOLDS_TUNING = 3 
#     RANDOM_STATE = 42
    
#     print(f"\n🚀 [Trial {trial.number}] 시작! (데이터 샘플링 중...)")
#     start_trial = time.time()

#     # --- 2. 데이터 샘플링 ---
#     try:
#         _, X_sample, _, y_sample = train_test_split(
#             X, y_encoded, test_size=SAMPLE_FRAC,
#             random_state=RANDOM_STATE + trial.number, stratify=y_encoded
#         )
#         X_sample = X_sample.reset_index(drop=True)
#         y_sample = y_sample.reset_index(drop=True) if hasattr(y_sample, 'reset_index') else y_sample
#     except Exception as e:
#         print(f"❌ Subsampling Error: {e}")
#         return 0.0
    
#     gc.collect()

#     params = {
#         'objective': 'multi:softprob',
#         'num_class': num_classes,
#         'eval_metric': 'mlogloss',
#         'eta': trial.suggest_float('eta', 0.01, 0.1, log=True),
#         'max_depth': trial.suggest_int('max_depth', 4, 10),
#         'subsample': trial.suggest_float('subsample', 0.6, 1.0),
#         'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
#         'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
#         'gamma': trial.suggest_float('gamma', 0.0, 5.0),
#         'lambda': trial.suggest_float('lambda', 0.1, 100.0, log=True),
#         'alpha': trial.suggest_float('alpha', 0.0, 10.0),
#         'tree_method': 'hist',
#         'device': 'cuda',
#         'random_state': RANDOM_STATE,
#         'nthread': -1
#     }

#     oof_preds_proba = np.zeros((len(X_sample), num_classes))
#     oof_labels = np.zeros(len(X_sample))
#     kf = StratifiedKFold(n_splits=N_FOLDS_TUNING, shuffle=True, random_state=RANDOM_STATE)
#     cat_cols = X_sample.select_dtypes(include=['object', 'category']).columns.tolist()

#     # --- 3. Fold 반복 ---
#     for fold, (train_idx, val_idx) in enumerate(kf.split(X_sample, y_sample)):
#         print(f"   Current Fold: {fold+1}/{N_FOLDS_TUNING}...", end=" ")
        
#         X_train_fold, X_val_fold = X_sample.iloc[train_idx].copy(), X_sample.iloc[val_idx].copy()
#         y_train_fold, y_val_fold = y_sample[train_idx], y_sample[val_idx]

#         # (1) Target Encoding
#         if cat_cols:
#             try:
#                 te = ce.TargetEncoder(cols=cat_cols, handle_missing='value', handle_unknown='value', smoothing=1.0)
#                 X_train_fold = te.fit_transform(X_train_fold, y_train_fold)
#                 X_val_fold = te.transform(X_val_fold)
#             except Exception as e: print(f"(TE Skip: {e})", end=" ")

#         # (2) SMOTE (여기가 오래 걸림)
#         try:
#             # print("SMOTE...", end=" ") # 너무 시끄러우면 주석 처리
#             smote = SMOTE(random_state=RANDOM_STATE + fold + trial.number, k_neighbors=5)
#             X_train_resampled, y_train_resampled = smote.fit_resample(X_train_fold, y_train_fold)
#         except:
#             X_train_resampled, y_train_resampled = X_train_fold, y_train_fold

#         X_train_resampled = X_train_resampled.astype(np.float32)
#         X_val_fold = X_val_fold.astype(np.float32)

#         # (3) XGBoost
#         # print("Train...", end=" ")
#         dtrain = xgb.DMatrix(X_train_resampled, label=y_train_resampled)
#         dval = xgb.DMatrix(X_val_fold, label=y_val_fold)
        
#         booster = xgb.train(
#             params=params, dtrain=dtrain, num_boost_round=300, # 라운드 수 줄임 (속도 향상)
#             evals=[(dval, 'eval')], early_stopping_rounds=30,
#             verbose_eval=False
#         )

#         val_preds = booster.predict(dval, iteration_range=(0, booster.best_iteration))
#         oof_preds_proba[val_idx] = val_preds
#         oof_labels[val_idx] = y_val_fold
        
#         print("✅ Done.") # Fold 완료 표시
#         del X_train_fold, X_val_fold, dtrain, dval, booster; gc.collect()

#     # --- 4. 결과 출력 ---
#     final_preds = np.argmax(oof_preds_proba, axis=1)
#     macro_f1 = f1_score(oof_labels, final_preds, average='macro', zero_division=0)
    
#     elapsed = time.time() - start_trial
#     print(f"   -> Trial {trial.number} 완료! Score: {macro_f1:.4f} (소요시간: {elapsed:.1f}초)")
    
#     return macro_f1

# # 🏆 실행
# print("🔄 Optuna 최적화 시작 (로그 출력 모드)...")
# study = optuna.create_study(direction='maximize')
# study.optimize(objective, n_trials=10) # 일단 10회만

# print("\n🏆 최적의 파라미터:", study.best_params)
# print("🏆 최고 Macro F1 점수:", study.best_value)

[I 2026-02-06 11:33:58,346] A new study created in memory with name: no-name-3f98a0ce-259d-4695-b628-1d1bafc4f79b


🔄 Optuna 최적화 시작 (로그 출력 모드)...

🚀 [Trial 0] 시작! (데이터 샘플링 중...)
   Current Fold: 1/3... ✅ Done.
   Current Fold: 2/3... ✅ Done.
   Current Fold: 3/3... 

[I 2026-02-06 11:35:30,181] Trial 0 finished with value: 0.4616319921851906 and parameters: {'eta': 0.05101828393625498, 'max_depth': 9, 'subsample': 0.736709985014893, 'colsample_bytree': 0.833797859056018, 'min_child_weight': 10, 'gamma': 3.986697747660632, 'lambda': 0.39255232615091845, 'alpha': 1.3779633633402721}. Best is trial 0 with value: 0.4616319921851906.


✅ Done.
   -> Trial 0 완료! Score: 0.4616 (소요시간: 91.8초)

🚀 [Trial 1] 시작! (데이터 샘플링 중...)
   Current Fold: 1/3... ✅ Done.
   Current Fold: 2/3... ✅ Done.
   Current Fold: 3/3... 

[I 2026-02-06 11:36:54,066] Trial 1 finished with value: 0.457117478209795 and parameters: {'eta': 0.04240585890226002, 'max_depth': 10, 'subsample': 0.7806578065218979, 'colsample_bytree': 0.9161337182060387, 'min_child_weight': 3, 'gamma': 4.859801889678062, 'lambda': 36.41054180189917, 'alpha': 1.5202325861847155}. Best is trial 0 with value: 0.4616319921851906.


...

✅ Done.
   -> Trial 7 완료! Score: 0.4564 (소요시간: 82.5초)

🚀 [Trial 8] 시작! (데이터 샘플링 중...)
   Current Fold: 1/3... ✅ Done.
   Current Fold: 2/3... ✅ Done.
   Current Fold: 3/3... 

[I 2026-02-06 11:46:28,014] Trial 8 finished with value: 0.4573527657478943 and parameters: {'eta': 0.08818330135875674, 'max_depth': 4, 'subsample': 0.7450580699067644, 'colsample_bytree': 0.7066637563333003, 'min_child_weight': 5, 'gamma': 3.8819478066458792, 'lambda': 65.86518804877579, 'alpha': 3.2279784892359995}. Best is trial 5 with value: 0.5250283406873293.


✅ Done.
   -> Trial 8 완료! Score: 0.4574 (소요시간: 61.2초)

🚀 [Trial 9] 시작! (데이터 샘플링 중...)
   Current Fold: 1/3... ✅ Done.
   Current Fold: 2/3... ✅ Done.
   Current Fold: 3/3... 

[I 2026-02-06 11:47:46,150] Trial 9 finished with value: 0.4549983672339241 and parameters: {'eta': 0.044163157513385815, 'max_depth': 7, 'subsample': 0.7354837808956604, 'colsample_bytree': 0.5166503467939775, 'min_child_weight': 4, 'gamma': 2.2854772263487, 'lambda': 45.772934866955744, 'alpha': 9.996963986647163}. Best is trial 5 with value: 0.5250283406873293.


✅ Done.
   -> Trial 9 완료! Score: 0.4550 (소요시간: 78.1초)

🏆 최적의 파라미터: {'eta': 0.0726332393055898, 'max_depth': 6, 'subsample': 0.9862407825133279, 'colsample_bytree': 0.5857635419003775, 'min_child_weight': 8, 'gamma': 0.18152204465888777, 'lambda': 0.14442660556652942, 'alpha': 2.459443414279423}
🏆 최고 Macro F1 점수: 0.5250283406873293


In [ ]:
# =============================================================================
# 🏆 [Step 2] 최종 학습 및 제출 파일 생성 (자동 연동)
# =============================================================================

# 1. 최적 파라미터 가져오기 (손으로 입력할 필요 X)
print("\n🔄 [시스템] 최적 파라미터 로드 중...")

try:
    # 방금 Optuna가 찾은 최고의 파라미터를 가져옵니다.
    best_params = study.best_params
    print(f"   ✅ Optuna 최적값 발견! (eta={best_params.get('eta', 'N/A'):.4f}, depth={best_params.get('max_depth', 'N/A')})")
    
    # XGBoost 필수 고정 파라미터 추가
    best_params.update({
        'objective': 'multi:softprob',
        'num_class': num_classes,
        'eval_metric': 'mlogloss',
        'tree_method': 'hist',
        'device': 'cuda',
        'nthread': -1,
        'random_state': 42
    })
    
except NameError:
    # 만약 Optuna를 안 돌리고 바로 이 코드를 실행했을 경우를 대비한 '비상용' 기본값
    print("   ⚠️ Optuna 실행 기록이 없습니다. 기본 파라미터로 실행합니다.")
    best_params = {
        'objective': 'multi:softprob',
        'num_class': num_classes,
        'eval_metric': 'mlogloss',
        'eta': 0.05, 
        'max_depth': 8,
        'tree_method': 'hist',
        'device': 'cuda'
    }

# 2. 최종 학습 실행 여부 결정
# MODE가 'PARAM_TUNING'일 때만 자동으로 100% 학습까지 이어가게 하거나,
# 아니면 무조건 실행하게 할 수도 있습니다. 여기선 질문하신 대로 '연결'해드립니다.

if MODE == 'PARAM_TUNING':
    print("\n🚀 [32GB Full Power] 전체 데이터(100%)로 최종 학습을 시작합니다!")
    
    # (1) 데이터 설정 (전체 데이터 X, y_encoded 사용)
    N_FOLDS_FINAL = 5
    kf = StratifiedKFold(n_splits=N_FOLDS_FINAL, shuffle=True, random_state=42)
    
    final_test_preds = np.zeros((len(X_test), num_classes))
    oof_preds = np.zeros((len(X), num_classes))
    
    start_time = time.time()
    
    # (2) 5-Fold 학습 루프
    for fold, (train_idx, val_idx) in enumerate(kf.split(X, y_encoded)):
        print(f"\n   [Fold {fold+1}/{N_FOLDS_FINAL}] 진행 중...", end=" ")
        
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y_encoded[train_idx], y_encoded[val_idx]
        
        # Target Encoding
        cat_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
        if cat_cols:
            te = ce.TargetEncoder(cols=cat_cols)
            X_train = te.fit_transform(X_train, y_train)
            X_val = te.transform(X_val)
            X_test_fold = te.transform(X_test.copy())
        else:
            X_test_fold = X_test.copy()

        # SMOTE (메모리 터질 수 있으니 조심)
        try:
            smote = SMOTE(random_state=42+fold, k_neighbors=5)
            X_train, y_train = smote.fit_resample(X_train, y_train)
        except: pass # 메모리 부족하면 패스
        
        # XGBoost Train
        dtrain = xgb.DMatrix(X_train, label=y_train)
        dval = xgb.DMatrix(X_val, label=y_val)
        dtest = xgb.DMatrix(X_test_fold)
        
        model = xgb.train(
            params=best_params, # ✅ 아까 찾은 최적값 자동 적용
            dtrain=dtrain,
            num_boost_round=2000, # 최종이니까 넉넉하게
            evals=[(dval, 'eval')],
            early_stopping_rounds=50,
            verbose_eval=False
        )
        
        # 예측
        oof_preds[val_idx] = model.predict(dval, iteration_range=(0, model.best_iteration))
        final_test_preds += model.predict(dtest, iteration_range=(0, model.best_iteration)) / N_FOLDS_FINAL
        
        # 메모리 정리
        del X_train, X_val, dtrain, dval, dtest, model; gc.collect()
        print("완료.")

    # (3) 제출 파일 생성
    print(f"\n✅ 최종 학습 완료! (소요시간: {(time.time() - start_time)/60:.1f}분)")
    
    submission = pd.DataFrame({
        'ID': X_test.index if 'ID' not in X_test.columns else X_test['ID'],
        'Segment': np.argmax(final_test_preds, axis=1)
    })
    submission.to_csv('final_submission_auto.csv', index=False)
    print("🎉 제출 파일 생성 완료: final_submission_auto.csv")

else:
    print("\n✋ [실험 종료] 파생변수 테스트 모드이므로 최종 학습은 건너뜁니다.")
    print("   -> 최종 학습을 하려면 맨 위 MODE를 'PARAM_TUNING'으로 바꿔주세요.")

In [ ]:
# import pandas as pd
# import numpy as np
# import xgboost as xgb
# import category_encoders as ce
# from imblearn.over_sampling import SMOTE
# from sklearn.model_selection import StratifiedKFold
# from sklearn.metrics import f1_score
# import gc
# import time

# # ====================================================
# # 1. 파라미터 및 설정
# # ====================================================
# try:
#     best_params = study.best_params
#     print(f"🏆 Optuna 최적 파라미터 로드 완료.")
# except:
#     print("⚠️ Optuna 결과가 없어 기본 파라미터를 사용합니다.")
#     best_params = {
#         # 'eta': 0.05, 'max_depth': 8, 'subsample': 0.8,
#         'colsample_bytree': 0.8, 'min_child_weight': 3,
#         'gamma': 1, 'lambda': 10, 'alpha': 1
#     }

# best_params.update({
#     'objective': 'multi:softprob',
#     'num_class': num_classes,
#     'eval_metric': 'mlogloss',
#     'tree_method': 'hist',
#     'device': 'cuda',
#     'nthread': -1,
#     'random_state': 42
# })

# # ====================================================
# # 2. 5-Fold 최종 학습 (Safety SMOTE 적용)
# # ====================================================
# N_FOLDS = 5
# kf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=42)

# final_test_preds = np.zeros((len(X_test), num_classes))
# oof_preds = np.zeros((len(X_final), num_classes))

# print(f"\n🚀 [32GB Mode] 최종 학습 시작 (SMOTE ON)...")
# start_time = time.time()

# cat_cols = X_final.select_dtypes(include=['object', 'category']).columns.tolist()

# for fold, (train_idx, val_idx) in enumerate(kf.split(X_final, y_encoded)):
#     fold_start = time.time()
#     print(f"\n🔄 [Fold {fold+1}/{N_FOLDS}] 데이터 분할 중...", end=" ")
    
#     X_train, X_val = X_final.iloc[train_idx].copy(), X_final.iloc[val_idx].copy()
#     y_train, y_val = y_encoded[train_idx], y_encoded[val_idx]
    
#     # (1) Target Encoding
#     if cat_cols:
#         te = ce.TargetEncoder(cols=cat_cols, handle_missing='value', handle_unknown='value', smoothing=1.0)
#         X_train = te.fit_transform(X_train, y_train)
#         X_val = te.transform(X_val)
#         X_test_fold = te.transform(X_test.copy()) # Test셋은 복사해서 변환
#     else:
#         X_test_fold = X_test.copy()
        
#     print("Target Encoding 완료...", end=" ")

#     # (2) SMOTE (메모리 안전장치 추가)
#     try:
#         # print("SMOTE 적용 시도...", end=" ")
#         smote = SMOTE(random_state=42 + fold, k_neighbors=5)
#         # SMOTE는 메모리를 많이 쓰므로 try-except로 감쌈
#         X_train, y_train = smote.fit_resample(X_train, y_train)
#         print(f"SMOTE 성공(Train: {len(X_train)}행)...", end=" ")
#     except Exception as e:
#         print(f"⚠️ SMOTE 메모리 부족으로 건너뜀({e})...", end=" ")
#         # 실패하면 원본 그대로 진행 (멈추지 않음)

#     # (3) 수치형 변환 (메모리 다이어트)
#     X_train = X_train.apply(pd.to_numeric, errors='coerce').fillna(0).astype(np.float32)
#     X_val = X_val.apply(pd.to_numeric, errors='coerce').fillna(0).astype(np.float32)
#     X_test_fold = X_test_fold.apply(pd.to_numeric, errors='coerce').fillna(0).astype(np.float32)
    
#     # (4) XGBoost 학습
#     dtrain = xgb.DMatrix(X_train, label=y_train)
#     dval = xgb.DMatrix(X_val, label=y_val)
#     dtest = xgb.DMatrix(X_test_fold)
    
#     print("학습 시작...")
#     model = xgb.train(
#         params=best_params,
#         dtrain=dtrain,
#         num_boost_round=1500, # SMOTE 데이터는 학습이 더 필요할 수 있어 라운드 증가
#         evals=[(dval, 'eval')],
#         early_stopping_rounds=50,
#         verbose_eval=False # 로그 너무 길면 끔
#     )
    
#     # (5) 예측 및 정리
#     oof_preds[val_idx] = model.predict(dval, iteration_range=(0, model.best_iteration))
#     final_test_preds += model.predict(dtest, iteration_range=(0, model.best_iteration)) / N_FOLDS
    
#     # 현재 Fold 점수 확인
#     fold_pred_labels = np.argmax(oof_preds[val_idx], axis=1)
#     fold_score = f1_score(y_val, fold_pred_labels, average='macro')
#     print(f"   -> Fold {fold+1} 완료. Score: {fold_score:.5f} (소요: {(time.time()-fold_start)/60:.1f}분)")
    
#     # 메모리 즉시 해제 (가장 중요)
#     del X_train, X_val, y_train, y_val, dtrain, dval, dtest, model, X_test_fold
#     gc.collect()

# # ====================================================
# # 3. 결과 저장
# # ====================================================
# print(f"\n✅ 전체 학습 완료! (총 소요: {(time.time() - start_time)/60:.1f}분)")

# # OOF 점수
# oof_pred_labels = np.argmax(oof_preds, axis=1)
# final_score = f1_score(y_encoded, oof_pred_labels, average='macro')
# print(f"🏆 Final OOF Macro F1 Score: {final_score:.5f}")

# # 제출 파일 생성
# test_pred_labels = np.argmax(final_test_preds, axis=1)
# submission_id = X_test.index if 'ID' not in X_test.columns else X_test['ID']

# submission = pd.DataFrame({
#     'ID': submission_id,
#     'Segment': test_pred_labels
# })

# submission.to_csv('submission_xgb_smote_32gb.csv', index=False)
# print("🎉 제출 파일 생성 완료: submission_xgb_smote_32gb.csv")

🏆 Optuna 최적 파라미터 로드 완료.

🚀 [32GB Mode] 최종 학습 시작 (SMOTE ON)...

🔄 [Fold 1/5] 데이터 분할 중... Target Encoding 완료... ⚠️ SMOTE 메모리 부족으로 건너뜀(Unable to allocate 5.00 GiB for an array with shape (1536865, 437) and data type float64)... 학습 시작...
   -> Fold 1 완료. Score: 0.68805 (소요: 42.1분)

🔄 [Fold 2/5] 데이터 분할 중... Target Encoding 완료... ⚠️ SMOTE 메모리 부족으로 건너뜀(Unable to allocate 5.01 GiB for an array with shape (1537527, 437) and data type float64)... 학습 시작...
   -> Fold 2 완료. Score: 0.71978 (소요: 19.7분)

🔄 [Fold 3/5] 데이터 분할 중... Target Encoding 완료... ⚠️ SMOTE 메모리 부족으로 건너뜀(Unable to allocate 25.0 GiB for an array with shape (7688205, 437) and data type float64)... 학습 시작...
   -> Fold 3 완료. Score: 0.68229 (소요: 23.8분)

🔄 [Fold 4/5] 데이터 분할 중... Target Encoding 완료... ⚠️ SMOTE 메모리 부족으로 건너뜀(Unable to allocate 25.0 GiB for an array with shape (7688205, 437) and data type float64)... 학습 시작...
   -> Fold 4 완료. Score: 0.67705 (소요: 23.9분)

🔄 [Fold 5/5] 데이터 분할 중... Target Encoding 완료... ⚠️ SMOTE 메모리 부족으로 건너뜀(Unab

In [6]:
import pandas as pd
import numpy as np

# 1. 파일 경로 설정 (사용자님의 PC 환경에 맞게 수정)
sample_path = r'C:\Users\CY2\github\DataScience\Credit Card\sample_submission.csv'
pred_file_path = r'C:\Users\CY2\github\DataScience\submission_xgb_smote_32gb.csv'

try:
    # 데이터 로드
    sample_sub = pd.read_csv(sample_path)
    user_pred = pd.read_csv(pred_file_path)

    print(f"✅ 파일을 성공적으로 불러왔습니다.")
    print(f"   - 샘플 행 수: {len(sample_sub)}")
    print(f"   - 예측 행 수: {len(user_pred)}")

    # 2. 숫자 세그먼트를 문자로 매핑 (0->A, 1->B, 2->C, 3->D, 4->E)
    # 만약 기존에 사용했던 LabelEncoder 순서가 다르다면 이 부분을 수정하세요.
    mapping = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E'}
    
    # 3. 데이터 매칭 및 변환
    # 샘플 파일의 행 수(100,000)만큼 예측 데이터에서 가져옵니다.
    target_preds = user_pred.iloc[:len(sample_sub)].copy()
    
    # 매핑 적용
    sample_sub['Segment'] = target_preds['Segment'].map(mapping)

    # 4. 결측치 확인 및 처리 (만약 매핑되지 않은 값이 있다면 'A'로 기본값 처리)
    if sample_sub['Segment'].isnull().any():
        print("⚠️ 매핑되지 않은 값이 있어 기본값 'A'로 채웁니다.")
        sample_sub['Segment'] = sample_sub['Segment'].fillna('A')

    # 5. 최종 저장
    save_path = r'C:\Users\CY2\github\DataScience\final_submission_recovered.csv'
    sample_sub.to_csv(save_path, index=False)

    print(f"\n🎉 최종 제출 파일이 생성되었습니다!")
    print(f"📍 저장 경로: {save_path}")
    print("\n[제출 파일 미리보기]")
    print(sample_sub.head())

except FileNotFoundError:
    print("❌ 에러: 지정된 경로에서 파일을 찾을 수 없습니다. 경로를 다시 확인해주세요.")
except Exception as e:
    print(f"❌ 오류 발생: {e}")

✅ 파일을 성공적으로 불러왔습니다.
   - 샘플 행 수: 100000
   - 예측 행 수: 600000

🎉 최종 제출 파일이 생성되었습니다!
📍 저장 경로: C:\Users\CY2\github\DataScience\final_submission_recovered.csv

[제출 파일 미리보기]
           ID Segment
0  TEST_00000       E
1  TEST_00001       E
2  TEST_00002       D
3  TEST_00003       E
4  TEST_00004       E
